In [ ]:
with open("requirements.txt", "w") as f:
    f.write("kfp==1.4.0\n")
    
!pip install -r requirements.txt  --upgrade --user

In [11]:
from typing import NamedTuple

import kfp
from kfp import dsl
from kfp.components import func_to_container_op, InputPath, OutputPath

from typing import NamedTuple
def mnist_func(log_folder:str) -> NamedTuple('Outputs', [('logdir', str)]):
    
    print('mnist_func:', log_folder)
    import tensorflow as tf
    import json
    mnist = tf.keras.datasets.mnist
    (x_train,y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train/255.0, x_test/255.0

    def create_model():
        return tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape = (28,28)),
            tf.keras.layers.Dense(512, activation = 'relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation = 'softmax')
        ])
    model = create_model()
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    import datetime
    import os
    log_dir = os.path.join(log_folder, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    model.fit(x=x_train, 
              y=y_train, 
              epochs=5, 
              validation_data=(x_test, y_test), 
              callbacks=[tensorboard_callback])

    print('At least tensorboard callbacks are correct')
    print('logdir:', log_dir)
    return ([log_dir])

def tensorboard_func(log_dir:str) -> NamedTuple('Outputs', [('mlpipeline_ui_metadata', 'UI_metadata')]):
    
    print('tensorboard_func:', log_dir)
    metadata = {
      'outputs' : [{
        'type': 'tensorboard',
        'source': log_dir,
      }]
    }
    import json
    return ([json.dumps(metadata)])


In [12]:
import kfp.dsl as dsl
import kfp.components as components

@dsl.pipeline(
   name='mnist pipeline',
   description='A pipeline to train a model on mnist dataset and start a tensorboard.'
)
def mnist_pipeline():

    log_folder = '/persist-log'
    pvc_name = "mypvc"
    
    vop = dsl.VolumeOp(
        name=pvc_name,
        resource_name="newpvc",
        size="10Gi",
        modes=dsl.VOLUME_MODE_RWM
    )
    mnist_op = func_to_container_op(
        func=mnist_func,
        base_image="tensorflow/tensorflow:2.0.0-py3",
    )
    tensorboard_op = func_to_container_op(
        func=tensorboard_func,
        base_image="python:3.7-slim",
    )
    
    mnist_task = mnist_op(log_folder).add_pvolumes({
        log_folder:vop.volume,
    })
    tenaorboard_task = tensorboard_op(mnist_task.outputs['logdir']).add_pvolumes({
        log_folder:vop.volume,
    })


In [13]:
kfp.compiler.Compiler().compile(mnist_pipeline, 'helloworld.zip')